In [1]:
import json
import numpy as np


In [2]:
with open("./released/sents.test", "r") as f:
    texts_test = f.read()
f.close()


In [3]:
with open("./released/model-file/transition_probability", "r") as f:
    A = json.load(f)
f.close()
with open("./released/model-file/observation_likelihood", "r") as f:
    B = json.load(f)
f.close()


In [4]:
paragraphs_test = texts_test.split("\n")[:-1]


In [5]:
p1 = paragraphs_test[0]


In [6]:
def get_tprob(A: dict, from_state: str, to_state: str):
    out = 0
    if to_state in A[from_state].keys():
        out = A[from_state][to_state]
    return out


def get_obsl(B: dict, state: str, word: str):
    out = 0
    if word in B[state].keys():
        out = B[state][word]
    return out


In [7]:
words_test = p1.split(" ")


In [8]:
# Intial State
word = words_test[0]
viterbi = np.array(
    [[get_tprob(A, "[START]", state) * get_obsl(B, state, word) for state in B.keys()]]
).T
backpoint = -np.ones((len(B.keys()), 1))
for word in words_test[1:]:
    vit_new_col = np.array(
        [
            [
                (
                    viterbi[:, -1]
                    * np.array(
                        [get_tprob(A, prev_state, state) for prev_state in B.keys()]
                    )
                    * get_obsl(B, state, word)
                ).max()
                for state in B.keys()
            ]
        ]
    ).T
    bp_new_col = np.array(
        [
            [
                (
                    viterbi[:, -1]
                    * np.array(
                        [get_tprob(A, prev_state, state) for prev_state in B.keys()]
                    )
                ).argmax()
                for state in B.keys()
            ]
        ]
    ).T
    viterbi = np.column_stack((viterbi, vit_new_col))
    backpoint = np.column_stack((backpoint, bp_new_col))

vit_last = (
    viterbi[:, -1]
    * np.array([get_tprob(A, prev_state, "[END]") for prev_state in B.keys()])
).max()
bp_last = (
    viterbi[:, -1]
    * np.array([get_tprob(A, prev_state, "[END]") for prev_state in B.keys()])
).argmax()


In [9]:
backtrace = [bp_last]
for j in range(len(words_test), 1, -1):
    backtrace.insert(0, int(backpoint[backtrace[0]][j - 1]))


In [10]:
tag_test = [list(B.keys())[j] for j in backtrace]


In [11]:
output = [a + "/" + b for a, b in zip(words_test, tag_test)]


In [13]:
paragraph_tagged = " ".join(output)


In [14]:
paragraphs_tagged = []


In [16]:
paragraphs_tagged.append(paragraph_tagged)


In [20]:
texts_tagged = "\n".join(paragraphs_tagged) + "\n"


In [23]:
with open("temp", "w") as f:
    f.write(texts_tagged)
f.close()
